In [ ]:
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
#CSV 파일 불러오기
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

train/test 데이터의 shape

In [ ]:
print('Shape of app_train', app_train.shape)
app_train.head()

In [ ]:
print('Shape of app_test', app_test.shape)
app_test.head()

>application_train EDA

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].value_counts() / app_train.shape[0]

In [ ]:

app_train['TARGET'].astype(int).plot.hist()

Target 데이터의 비율을 확인해보았을 때 0이 지나치게 많아 불균형하다는 것을 확인할 수 있음

**결측값 확인**

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
missing_data(app_train).head(20)

In [ ]:
missing_data(app_test).head(20)

In [ ]:
#이상값 확인
app_train['DAYS_EMPLOYED'].describe()

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment');

In [ ]:
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [ ]:
plt.figure(figsize=(6,4))
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()
#이상값 없음

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

**상관관계 찾기**

In [ ]:
correlations = app_train.corr()['TARGET'].sort_values()
print('긍정적:', correlations.tail(15))
print('부정적:', correlations.head(15))

In [ ]:
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

In [ ]:
#연령 분포를 연도 단위로 표시
plt.style.use('fivethirtyeight')
plt.hist(app_train['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count');

In [ ]:
plt.figure(figsize = (10, 8))

sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

In [ ]:
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)